In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [111]:
#Loading data as df (dataframe)
df = pd.read_excel('Investor Report-06_21_2025.xlsx', sheet_name=0)
df.head(2)

,Case Name,Account Name,State,Rate,Provider Client,DOL,Subject,Name of Procedure,Case Stage,Stage Chg Dt,...,Inv 1 Pd Dt,Inv 1 Pd $,Inv 2 Pd Dt,Inv 2 Pd $,Inv 3 Pd Dt,Inv 3 Pd $,Funding Status,Case Notes,Lien Notes,Case Status
0,Kami Orton,ALLRED VOGT & STUART,NaN,NaN,Durango Surgery Center,NaN,Surgery 1,NaN,Omni Purchase,09/21/2023,...,NaN,0.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,Case was transferred per Rachelle \ndeal dire...
1,Chaves Still,Baric Law,California,NaN,Precision Medical Products,03/23/2023,Med Device 1,NaN,PENDING,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,Declined,Mgr: Jonathan 833-467-2022,Premise,VETTING \nsent vetting emailwith new bill-HD \...


In [112]:
# Antes de eliminar variables irrelevantes eliminamos los registros que no tienen fecha de pago
df = df.dropna(subset=['Pay Dt'])

In [113]:
# Data type casting (corrections)
categorical_columns = ['Account Name','Provider Client','Funding Status','Subject']
df[categorical_columns] = df[categorical_columns].astype('category')

numerical_columns = ['Bill Amt', 'Purch Amt','Pay Amt']
df[numerical_columns] = df[numerical_columns].astype('float')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4240 entries, 26 to 8271
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Case Name          4240 non-null   object  
 1   Account Name       4240 non-null   category
 2   State              74 non-null     object  
 3   Rate               828 non-null    object  
 4   Provider Client    4228 non-null   category
 5   DOL                3315 non-null   object  
 6   Subject            4240 non-null   category
 7   Name of Procedure  3528 non-null   object  
 8   Case Stage         4240 non-null   object  
 9   Stage Chg Dt       4234 non-null   object  
 10  DOS                4227 non-null   object  
 11  Batch Dt           4204 non-null   object  
 12  Bill Amt           4239 non-null   float64 
 13  Purch Amt          4239 non-null   float64 
 14  Purch Dt           4219 non-null   object  
 15  Investor 1         4238 non-null   object  
 16  Port 1    

In [114]:
df['Account Name'] = (
    df['Account Name']
    .astype(str)
    .str.strip()
    # .str.lower()
    .str.replace(r'\s+', ' ', regex=True)          # Espacios múltiples → uno
    .str.replace(r'[^\w\s&]', '', regex=True)      # Eliminar símbolos raros (excepto &)
    .str.strip()
)
# Creación de columna de limpieza
df['AccountName_Clean'] = np.nan

In [115]:
unique_clean = []

from rapidfuzz import fuzz

for firm in df['Account Name'].unique():
    clean = firm.strip()
    match_found = False
    for ref in unique_clean:
        if fuzz.ratio(clean, ref) > 80:   # Umbral de similitud
            match_found = True
            df.loc[df['Account Name'] == firm, 'AccountName_Clean'] = ref
            break
    if not match_found:
        unique_clean.append(clean)
        df.loc[df['Account Name'] == firm, 'AccountName_Clean'] = clean

/tmp/ipykernel_13413/3216805034.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A&T Legal Group' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Account Name'] == firm, 'AccountName_Clean'] = clean


In [116]:
# --- Reemplazar columna original por la limpia ---
df['Account Name'] = df['AccountName_Clean']
df = df.drop('AccountName_Clean', axis=1)

print(list(df['Account Name'].unique()))

['A&T Legal Group', 'Aaron Law Group', 'Accident Fighters APC', 'Accident Lawyers Group Pros', 'Ace Law Firm  California', 'Ace Law Group', 'ACTS Law Firm', 'Adamson Ahdoot LLP', 'Agwara Law', 'Alexandroff Law Group', 'Allen Injury Law LLC', 'Allison Law Firm', 'Angulo Law Group', 'Anthony Paglia', 'Ashton & Price LLP', 'Atkinson Watkins & Hoffman Attorneys', 'Avery Injury Law', 'Avrek Law', 'Ayala Morgan & Buzzard', 'BANAFSHE LAW FIRM PC', 'Baradat & Paboojian', 'BD & J Law Firm', 'Benjamin Nadig Law', 'Benson Allred Injury Law', 'Berg Law  California', 'Bertoldo Carter Smith & Mullen Attorneys', 'Beverly Law', 'Bighorn Law  Las Vegas', 'Blackburn Wirth Injury Team', 'Boley & Aldabbagh Attorneys', 'Bonakdar Law Firm', 'Boohoff Law', 'Bruning Law Firm', 'Burger Law', 'Burris & Thomas Law', 'Caffey Law Firm', 'California Accident Law Firm LOP', 'Call Jacob  The Law Offices of Jacob Emrani', 'Cardenas Law Group', 'Carpenter & Zuckerman', 'CARTER WOLDEN CURTIS LLP', 'Christiansen Trial La

In [117]:
df = df[df['Account Name'].map(df['Account Name'].value_counts()) > 6]
print(list(df['Account Name'].unique()))

['Ace Law Group', 'Angulo Law Group', 'Atkinson Watkins & Hoffman Attorneys', 'BD & J Law Firm', 'Benjamin Nadig Law', 'Benson Allred Injury Law', 'Blackburn Wirth Injury Team', 'Cardenas Law Group', 'David W Fassett Personal Injury Law', 'Dimopoulos Injury Law', 'ER Injury Attorneys', 'Fuller Law Practice', 'G Dallas Horton & Associates', 'Goldberg Injury Law', 'Jacoby & Meyers  CA', 'Ladah Law', 'Lalezary Law Firm  CA Law Brothers', 'Law Office of Arash Khorsandi', 'Law Office of Stephen Reid', 'Law Office of Victor M Cardoza', 'Lawrence C Hill & Associates', 'Lloyd Baker Injury Attorneys', 'Mullins & Trenchak Law', 'Muslusky Law', 'Naqvi Injury Law', 'Neal Hyman Law', 'Nwogbe Law Group', 'Parke Law Firm', 'RRCC Firm', 'Sidell Injury Law', 'Valarie Fujii Law Offices', 'Vannah & Vannah Law Firm', 'VC2 Law', 'Willoughby Shulman Injury Lawyers', 'Wilshire Law Firm  CA', 'Yan Kenyon Law']


In [118]:
df = df[df['Provider Client'].map(df['Provider Client'].value_counts()) > 3]
print(list(df['Provider Client'].unique()))

['Pueblo Medical Imaging', 'LVR', 'Durango Surgery Center', 'Valley Health System', 'Parkway Surgery Center', 'Suarez Physical Therapy', 'Jackson Physical Therapy', 'Precision Medical Products', 'Simpson Chiropractic', 'Ascension Medical Center', 'OpenSided MRI of Las Vegas', 'LVC Surgery Center', 'Surgical Arts Centre']


In [119]:
df = df[df['Subject'].map(df['Subject'].value_counts()) > 3]
print(list(df['Subject'].unique()))

['Lien1', 'Lien2', 'imaging 1', 'imaging 2', 'imaging 3', 'imaging1', 'imaging2', 'pain1', 'imaging3', 'Imaging1', 'Imaging2', 'Imaging 1', 'Imaging 2', 'Imaging 3', 'Imaging 5', 'Imaging 4', 'Pain 1', 'Imaging 1 ', 'Imaging 6', 'imaging4', 'pt1', 'PT 1', 'Physical 1', 'physicaltherapy 1', 'Med Device 1', 'chiro 1', 'chiro1', 'device1', 'Med Device 2', 'imaging5', 'imaging6', 'Imaging3', 'Imaging4', 'Imaging 7', 'Imaging 8', 'Imaging 9', 'Imaging 11', 'physicaltherapy1', 'imaging 4', 'imaging 5', 'imaging 6', 'Funding1', 'Lien3', 'Lien4', 'Imaging5', 'Imaging6', 'Pain1', 'imaging 7', 'imaging 8', 'imaging 9', 'physical therapy 1', 'physical therapy', 'Imaging 10', 'Surgery 1', 'Med Device 1 ', 'Lien5']


In [120]:
len(list(df['Account Name'].unique()))
# Quedaron un total de 251 categorías

36

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3698 entries, 45 to 8231
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Case Name          3698 non-null   object  
 1   Account Name       3698 non-null   object  
 2   State              13 non-null     object  
 3   Rate               646 non-null    object  
 4   Provider Client    3698 non-null   category
 5   DOL                2909 non-null   object  
 6   Subject            3698 non-null   category
 7   Name of Procedure  3354 non-null   object  
 8   Case Stage         3698 non-null   object  
 9   Stage Chg Dt       3692 non-null   object  
 10  DOS                3695 non-null   object  
 11  Batch Dt           3686 non-null   object  
 12  Bill Amt           3697 non-null   float64 
 13  Purch Amt          3697 non-null   float64 
 14  Purch Dt           3680 non-null   object  
 15  Investor 1         3698 non-null   object  
 16  Port 1    

In [122]:
df.head()

,Case Name,Account Name,State,Rate,Provider Client,DOL,Subject,Name of Procedure,Case Stage,Stage Chg Dt,...,Inv 1 Pd Dt,Inv 1 Pd $,Inv 2 Pd Dt,Inv 2 Pd $,Inv 3 Pd Dt,Inv 3 Pd $,Funding Status,Case Notes,Lien Notes,Case Status
45,Derek Bell,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,NaN,Lien1,MRI Joint Upper Extremity wo Dye,Completed,01/13/2022,...,06/21/2021,825.0,NaN,0.0,NaN,0.0,Purchased,NaN,"02,20 Settled","2/20/20: Settled per case mgr, \nPaid complete..."
46,Derek Bell,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,NaN,Lien2,MRI Cervical Spine wo Contrast,Completed,01/13/2022,...,06/21/2021,825.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,"2/20/20: Settled per case mgr, \nPaid complete..."
53,Rhonda Anderson,Ace Law Group,NaN,NaN,LVR,12/10/2018,imaging 1,MRI Cervical Spine wo Contrast,Completed,01/30/2023,...,NaN,0.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,In Litigation \n
54,Rhonda Anderson,Ace Law Group,NaN,NaN,LVR,12/10/2018,imaging 2,MRI Lumbar Spine wo Contrast,Completed,01/30/2023,...,NaN,0.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,In Litigation \n
55,Rhonda Anderson,Ace Law Group,NaN,NaN,LVR,12/10/2018,imaging 3,MRI Thoracic Spine wo Contrast,Completed,01/30/2023,...,NaN,0.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,In Litigation \n


In [123]:
df = df.dropna(subset=['Purch Amt'])
df = df[df['Purch Amt'] != 0]

In [124]:
df = df[df['Purch Amt'] != df['Pay Amt']]

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3673 entries, 45 to 8231
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Case Name          3673 non-null   object  
 1   Account Name       3673 non-null   object  
 2   State              13 non-null     object  
 3   Rate               645 non-null    object  
 4   Provider Client    3673 non-null   category
 5   DOL                2887 non-null   object  
 6   Subject            3673 non-null   category
 7   Name of Procedure  3338 non-null   object  
 8   Case Stage         3673 non-null   object  
 9   Stage Chg Dt       3667 non-null   object  
 10  DOS                3671 non-null   object  
 11  Batch Dt           3663 non-null   object  
 12  Bill Amt           3673 non-null   float64 
 13  Purch Amt          3673 non-null   float64 
 14  Purch Dt           3657 non-null   object  
 15  Investor 1         3673 non-null   object  
 16  Port 1    

In [126]:
df.to_excel("Portfolio Puma MGM.xlsx", index=False)